# 2-D Cross-Section Meshing for Mode Analysis

This notebook demonstrates using `ModalSim` to slice a 3-D CPW (coplanar waveguide) geometry with a cutting plane and produce a 2-D triangular mesh. The resulting `.msh` file can be loaded into eigenmode solvers such as [femwell](https://github.com/HelgeGehwornis/femwell).

**Requirements:**

- IHP PDK: `uv pip install ihp-gdsfactory`

### Load a pcell from IHP PDK

In [ ]:
import gdsfactory as gf
from ihp import LAYER, PDK

PDK.activate()


@gf.cell
def gsg_electrode(
    length: float = 300,
    s_width: float = 20,
    g_width: float = 40,
    gap_width: float = 15,
    layer=LAYER.TopMetal2drawing,
) -> gf.Component:
    """
    Create a GSG (Ground-Signal-Ground) electrode.

    Args:
        length: horizontal length of the electrodes
        s_width: width of the signal (center) electrode
        g_width: width of the ground electrodes
        gap_width: gap between signal and ground electrodes
        layer: layer for the metal
    """
    c = gf.Component()

    # Top ground electrode
    r1 = c << gf.c.rectangle((length, g_width), centered=True, layer=layer)
    r1.move((0, (g_width + s_width) / 2 + gap_width))

    # Center signal electrode
    _r2 = c << gf.c.rectangle((length, s_width), centered=True, layer=layer)

    # Bottom ground electrode
    r3 = c << gf.c.rectangle((length, g_width), centered=True, layer=layer)
    r3.move((0, -(g_width + s_width) / 2 - gap_width))

    # Add ports at the gaps
    c.add_port(
        name="P1",
        center=(-length / 2, -(s_width + gap_width) / 2),
        width=gap_width,
        orientation=0,
        port_type="electrical",
        layer=layer,
    )

    c.add_port(
        name="P2",
        center=(-length / 2, (s_width + gap_width) / 2),
        width=gap_width,
        orientation=0,
        port_type="electrical",
        layer=layer,
    )

    c.add_port(
        name="P3",
        center=(length / 2, (s_width + gap_width) / 2),
        width=gap_width,
        orientation=180,
        port_type="electrical",
        layer=layer,
    )

    c.add_port(
        name="P4",
        center=(length / 2, -(s_width + gap_width) / 2),
        width=gap_width,
        orientation=180,
        port_type="electrical",
        layer=layer,
    )

    return c


c = gsg_electrode()
cc = c.copy()
cc.draw_ports()
cc

### Configure 2-D cross-section with ModalSim

In [ ]:
from gsim.palace import ModalSim

# Create modal simulation object
sim = ModalSim()

# Set output directory
sim.set_output_dir("./modal-sim-cpw")

# Set the component geometry
sim.set_geometry(c)

# Configure layer stack from active PDK
sim.set_stack(substrate_thickness=2.0, air_above=300.0)

# Define a cutting plane at y = 0 (XZ cross-section through the centre)
sim.set_cross_section(x=0.0)

# Validate configuration
print(sim.validate_config())

In [ ]:
# Generate the 2-D cross-section mesh
sim.mesh(preset="default")

In [ ]:
# Plot the 2-D cross-section mesh
sim.plot_mesh(show_groups=["passive", "SiO2"], style="solid", interactive=True)

### Load the mesh in femwell

The generated `.msh` file can be loaded directly into femwell for eigenmode analysis.

In [ ]:
from pathlib import Path

mesh_path = Path("./modal-sim-cpw/palace.msh")
print(f"Mesh saved to: {mesh_path.resolve()}")
print(f"File size: {mesh_path.stat().st_size / 1024:.1f} KB")